In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
import random
from tokenizer import get_files, get_threads

files = get_files("./data")
threads = [x for x in get_threads(files)]
threads[0]

['Close', 'Máy đẹp, thêm hình được không thớt', 'Fix thêm hơm bạn']

In [3]:
def get_convo(threads):
#     SEP_TOKEN = " <SEP> "
    SEP_TOKEN = ". "
    
    for thread in threads:
        if len(thread) < 2:
            continue
        yield SEP_TOKEN.join(thread)


random.shuffle(threads)
convo_generator = get_convo(threads)
next(convo_generator)

'Combo Main Giga H81M-DS2 (có chặn)– CPU G3450: 800k (SL 6) Combo Main Biostar H61MGV3 (itx) – G2030 – Ram 2G : 600k Combo Main Asus P8H61 – CPU Celeron G550 – Ram 4G (2x2) : 650k Main lẻ Giga H81M-DS2 (không chặn): 500k (SL 3) hoặc trade main Asus H81M-D hoặc E GD HN (Khu vực cầu Nguyễn An Ninh hoặc Khuất duy Tiến) Hoặc Bank chuyển ship.. Main: giga Z390UD bh 13/06/2022: 1tr9 2 Chân màn dell 2312 150k/1. Mua cả 250k ĐC: 29 nguyễn bỉnh khiêm giờ hành chính sđt/zalo: 0948585179'

In [4]:
convo_generator = get_convo(threads)
all_convo = [x for x in convo_generator]
# all_convo = [x for thread in threads for x in thread if len(x) > 10]
random.shuffle(all_convo)

print("Number of threads:", len(all_convo))

Number of threads: 205340


In [5]:
all_convo = random.sample(all_convo, 10000)
print("Number of threads:", len(all_convo))

Number of threads: 10000


## Hugging Face

In [5]:
import numpy as np
import torch

In [6]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file="voz_tokenizer.json",
    unk_token="<UNK>",
    pad_token="<PAD>",
    bos_token="<BOS>",
    eos_token="<EOS>",
)

VOCAB_SIZE = tokenizer.vocab_size
VOCAB_SIZE

15000

In [7]:
encodings = tokenizer(all_convo[0])

print("input", all_convo[0].split()[:10])
for k,v in encodings.items():
    print(k,v[:10],"...")
print("tokens", tokenizer.convert_ids_to_tokens(encodings.input_ids[:10]))

input ['1m4.', '3c', 'gskill', 'ripjaws', 'Z', '4gb', '1600', 'cas9', '300/1', '2c']
input_ids [8100, 24, 18, 23, 71, 75, 8913, 13975, 8503, 13966] ...
token_type_ids [0, 0, 0, 0, 0, 0, 0, 0, 0, 0] ...
attention_mask [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] ...
tokens ['1m', '4', '.', '3', 'c', 'g', 'skill', 'rip', 'ja', 'ws']


In [8]:
lengths = [len(tokenizer(x).input_ids) for x in random.sample(all_convo, 100)]
lengths.sort()
lengths[-10:]

[3002, 3160, 3407, 3685, 4066, 4753, 5185, 7272, 8202, 16337]

In [9]:
samples = random.sample(all_convo, 16)

In [10]:
sequence_length = 2**15
sequence_length

32768

In [11]:
class VozDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, i):
        encodings = self.tokenizer(
            self.texts[i], 
            padding="max_length", 
            truncation=True,
            max_length=self.max_length,
            return_token_type_ids=False
        )
        encodings["labels"] = encodings["input_ids"]
        return {k:torch.tensor(v) for k,v in encodings.items()}

dataset = VozDataset(samples, tokenizer, sequence_length)

print(dataset[0])
print(dataset[:4])

{'input_ids': tensor([7650, 6436, 5896,  ...,    1,    1,    1]), 'attention_mask': tensor([1, 1, 1,  ..., 0, 0, 0]), 'labels': tensor([7650, 6436, 5896,  ...,    1,    1,    1])}
{'input_ids': tensor([[ 7650,  6436,  5896,  ...,     1,     1,     1],
        [ 6407,  6038,  6668,  ...,     1,     1,     1],
        [ 7437,  6554,  5937,  ...,     1,     1,     1],
        [11083,  5929,  5874,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([[ 7650,  6436,  5896,  ...,     1,     1,     1],
        [ 6407,  6038,  6668,  ...,     1,     1,     1],
        [ 7437,  6554,  5937,  ...,     1,     1,     1],
        [11083,  5929,  5874,  ...,     1,     1,     1]])}


In [12]:
class VozDataset(torch.utils.data.Dataset):
    def __init__(self, texts, tokenizer, max_length=float("inf")):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, i):
        encodings = self.tokenizer(
            self.texts[i], 
            return_token_type_ids=False
        )
        if len(encodings.input_ids) > self.max_length:
            return encodings.input_ids[:self.max_length]
        else:
            return encodings.input_ids

class VozCollator():
    def __init__(self, align_by, max_length, pad_value=0):
        self.align_by = align_by
        self.max_length = max_length
        self.pad_value = pad_value
    
    def __call__(self, batch):
        length = max([len(x) for x in batch])
        length = min(self.max_length, length)
        length = np.ceil(length/self.align_by).astype(int) * self.align_by   # align to chunk size
        
        attention_mask = [[1]*len(x) + [0]*(length-len(x)) if len(x) < length else [1]*length for x in batch]
        input_ids = [x + [self.pad_value]*(length-len(x)) if len(x) < length else x[:length] for x in batch]

        output = {
            "input_ids": torch.tensor(input_ids),
            "labels": torch.tensor(input_ids),
            "attention_mask": torch.tensor(attention_mask),
        }
        
        return output

dataset = VozDataset(samples, tokenizer, max_length=sequence_length)
collator = VozCollator(64, 1024, tokenizer.pad_token_id)
batch = collator(dataset[:4])
batch

{'input_ids': tensor([[ 7650,  6436,  5896,  ...,     1,     1,     1],
         [ 6407,  6038,  6668,  ...,     1,     1,     1],
         [ 7437,  6554,  5937,  ...,     1,     1,     1],
         [11083,  5929,  5874,  ...,  6183, 10793,  6452]]),
 'labels': tensor([[ 7650,  6436,  5896,  ...,     1,     1,     1],
         [ 6407,  6038,  6668,  ...,     1,     1,     1],
         [ 7437,  6554,  5937,  ...,     1,     1,     1],
         [11083,  5929,  5874,  ...,  6183, 10793,  6452]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 1, 1, 1]])}

In [13]:
random.shuffle(all_convo)
# train_size = int(len(all_convo)*0.95)
train_size = len(all_convo) - 1000

train_convo = all_convo[:train_size]
test_convo = all_convo[train_size:]

print("Train threads:", len(train_convo))
print("Test threads:", len(test_convo))

Train threads: 204340
Test threads: 1000


In [14]:
train_dataset = VozDataset(train_convo, tokenizer, sequence_length)
test_dataset = VozDataset(test_convo, tokenizer, sequence_length)

From https://huggingface.co/transformers/master/model_doc/reformer.html#axial-positional-encodings

<blockquote>
In practice, the parameter config.axial_pos_embds_dim is set to a tuple (d1,d2) which sum has to be equal to config.hidden_size and config.axial_pos_shape is set to a tuple (n1s,n2s) which product has to be equal to config.max_embedding_size, which during training has to be equal to the sequence length of the input_ids.
</blockquote>

From https://huggingface.co/transformers/master/model_doc/reformer.html#training

<blockquote>
During training, we must ensure that the sequence length is set to a value that can be divided by the least common multiple of config.lsh_chunk_length and config.local_chunk_length and that the parameters of the Axial Positional Encodings are correctly set as described above. Reformer is very memory efficient so that the model can easily be trained on sequences as long as 64000 tokens.
</blockquote>

In [15]:
from transformers import ReformerModelWithLMHead, ReformerConfig

config = {
#     "attention_head_size": 64,
#     "attn_layers": ["local", "lsh", "local", "lsh", "local", "lsh"],
#     "axial_pos_embds": True,
#     "sinusoidal_pos_embds": False,
#     "axial_pos_embds_dim": [64, 192],
    "axial_pos_shape": [2**7, 2**8],
    "eos_token_id": tokenizer.eos_token_id,
    "pad_token_id": tokenizer.pad_token_id,
    
#     "lsh_attn_chunk_length": 64,
#     "local_attn_chunk_length": 64,
    
#     "feed_forward_size": 512,
#     "hidden_act": "relu",
#     "hidden_size": 256,
    
    "is_decoder": True,
    "max_position_embeddings": sequence_length,
    "num_attention_heads": 2,
#     "num_buckets": [64, 128],
#     "num_buckets": None,
#     "num_hashes": 1,
    "vocab_size": VOCAB_SIZE,
#     "lsh_attention_probs_dropout_prob": 0.0,
#     "lsh_num_chunks_before": 1,
#     "lsh_num_chunks_after": 0,
#     "local_num_chunks_before": 1,
#     "local_num_chunks_after": 0,
#     "local_attention_probs_dropout_prob": 0.025,
#     "hidden_dropout_prob": 0.025,
}

config = ReformerConfig(
    axial_pos_embds=False,
#     axial_pos_shape=[2**7, 2**8],
    num_buckets=[32, 32],
    vocab_size=VOCAB_SIZE,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    is_decoder=True,
    max_position_embeddings=sequence_length,
#     num_attention_heads=2,
)
model = ReformerModelWithLMHead(config)
del model

In [16]:
def compute_metrics(pred):
    non_padded_indices = (pred.label_ids != -100)
    
    labels = pred.label_ids[..., 1:][non_padded_indices[..., 1:]]
    pred = np.argmax(pred.predictions[:, :-1], axis=-1)[non_padded_indices[..., :-1]]
    acc = np.mean(np.asarray(pred == labels), dtype=np.float)

    return {"accuracy": acc}

In [17]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./",
    num_train_epochs=5,
    do_train=True,
    do_eval=True,
    
    fp16=True,
    fp16_opt_level="O2",
#     gradient_accumulation_steps=8,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_accumulation_steps=1,
#     prediction_loss_only=True,
    
    learning_rate=1e-3,
    warmup_steps=1000,
    weight_decay=0.01,
    
    save_steps=1000,
    save_total_limit=10,
    
    evaluation_strategy="steps",
    eval_steps=1000,
    
#     logging_dir='./logs',
    logging_steps=10,
)

In [18]:
try:
    del model
except:
    pass

In [ ]:
model = ReformerModelWithLMHead(config)
model.train()

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator=collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()
trainer.save_model("voz_model")
del model

/home/students/student11_5/miniconda3/envs/vn-chatbot/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


Step,Training Loss,Validation Loss,Accuracy
1000,4.322900,5.255658,0.436373
2000,nan,nan,0.000000


In [21]:
model.eval()

output = model.generate(
    **tokenizer("xin chào tôi tên là Trần Nguyễn Mạnh Thiên", return_tensors="pt").to(model.device),
    max_length=100,
    do_sample=True,
    temperature=0.3,
    top_k=10,
#     bad_words_ids=[tokenizer("").input_ids]
)
print(tokenizer.decode(output[0]))

xin chào tôi tên là Trần Nguyễn Mạnh Thiên. <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>.,..,... <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


# Trax

In [ ]:
import numpy as np
import trax

output = trax.data.tokenize(iter([sample]), vocab_type="sentencepiece", vocab_file="voz_out.model", vocab_dir=".")
output = list(output)

restored = trax.data.detokenize(output[0], vocab_type="sentencepiece", vocab_file="voz_out.model", vocab_dir=".")

print(output)
print(restored)

In [ ]:
val_size = len(all_convo) // 20

train_convo = all_convo[:-val_size]
val_convo = all_convo[-val_size:]

print("Train size:", len(train_convo))
print("Val size:", len(val_convo))

In [ ]:
def stream(data):
    # loop over the entire data
    while True:
        # get a random element
        d = random.choice(data)
        
        # yield a tuple pair of identical values 
        # (i.e. our inputs to the model will also be our targets during training)
        yield (d, d)

In [ ]:
VOCAB_DIR = "."
VOCAB_FILE = "voz_out.model"

# trax allows us to use combinators to generate our data pipeline
data_pipeline = trax.data.Serial(
    # randomize the stream
    trax.data.Shuffle(),
    
    # tokenize the data
    trax.data.Tokenize(
        vocab_type="sentencepiece",
        vocab_dir=VOCAB_DIR,
        vocab_file=VOCAB_FILE
    ),
    
    # filter too long sequences
    trax.data.FilterByLength(2048),
    
    # bucket by length
    trax.data.BucketByLength(
        boundaries=[32,128,512,2048],
        batch_sizes=[256,64,16,4,1]
    ),
    
    # add loss weights but do not add it to the padding tokens (i.e. 0)
    trax.data.AddLossWeights(id_to_mask=0)
)

# apply the data pipeline to our train and eval sets
train_stream = data_pipeline(stream(train_convo))
eval_stream = data_pipeline(stream(val_convo))

In [ ]:
# the stream generators will yield (input, target, weights). let's just grab the input for inspection
inp, _, _ = next(train_stream)

# print the shape. format is (batch size, token length)
print("input shape: ", inp.shape)

# detokenize the first element
print(trax.data.detokenize(inp[0], vocab_type="sentencepiece", vocab_dir=VOCAB_DIR, vocab_file=VOCAB_FILE))

In [ ]:
def get_reformer(vocab_size=10000, n_layers=2, mode="train", attention_type=trax.layers.SelfAttention):
    model = trax.models.reformer.ReformerLM(
        vocab_size=vocab_size,
        n_layers=n_layers,
        mode=mode,
        attention_type=attention_type
    )
    return model

model = get_reformer()
print(model)
del(model)

In [ ]:
lr_schedule = trax.lr.warmup_and_rsqrt_decay(
    n_warmup_steps=1000, 
    max_value=0.01
)

train_task = trax.supervised.training.TrainTask(            
    labeled_data=train_stream,
    loss_layer=trax.layers.CrossEntropyLoss(),
    optimizer=trax.optimizers.Adam(0.01),
    lr_schedule=lr_schedule,
    n_steps_per_checkpoint=10
)

eval_task = trax.supervised.training.EvalTask(                      
    labeled_data=eval_stream,
    metrics=[trax.layers.CrossEntropyLoss(), trax.layers.Accuracy()]
)

loop = trax.supervised.training.Loop(
    get_reformer(),
    train_task,
    eval_tasks=[eval_task],
    output_dir="./model/"
)

In [ ]:
loop.run(10)

In [ ]:
def attention(*args, **kwargs):
    kwargs['predict_mem_len'] = 120
    kwargs['predict_drop_len'] = 120
    return trax.layers.SelfAttention(*args, **kwargs)

model = get_reformer(
    mode='predict',
    attention_type=attention,
)

shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)

model.init_from_file(
    './model/model.pkl.gz',
    weights_only=True, 
    input_signature=shape11)

STARTING_STATE = model.state